In [339]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

import plotly.graph_objs as go
import plotly.offline as pyoff
import statsmodels.formula.api as smf

In [340]:
#df = pd.read_csv("data/train.csv")
df = pd.read_excel('itService_day.xlsx')
df['RQST_DT'] = df['RQST_DT'].astype(str)
df['RQST_DT'] = pd.to_datetime(df['RQST_DT'])
df

,RQST_DT,CNT
0,2014-01-02,5
1,2014-01-03,4
2,2014-01-06,8
3,2014-01-07,6
4,2014-01-08,2
...,...,...
1731,2021-12-24,2
1732,2021-12-28,6
1733,2021-12-29,3
1734,2021-12-30,5


In [341]:
#create a new dataframe to model the difference
df_diff = df.copy()
#add previous sales to the next row
df_diff['prev_CNT'] = df_diff['CNT'].shift(1)
#drop the null values and calculate the difference
df_diff = df_diff.dropna()
df_diff['diff'] = (df_diff['CNT'] - df_diff['prev_CNT'])
df_diff.head(10)


,RQST_DT,CNT,prev_CNT,diff
1,2014-01-03,4,5.0,-1.0
2,2014-01-06,8,4.0,4.0
3,2014-01-07,6,8.0,-2.0
4,2014-01-08,2,6.0,-4.0
5,2014-01-09,2,2.0,0.0
6,2014-01-10,9,2.0,7.0
7,2014-01-13,7,9.0,-2.0
8,2014-01-14,3,7.0,-4.0
9,2014-01-15,4,3.0,1.0
10,2014-01-16,8,4.0,4.0


In [342]:
df_diff = df_diff[df['RQST_DT'].between('2021-01-01', '2021-12-31')]

C:\Users\admin\AppData\Local\Temp/ipykernel_9736/1483239927.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [343]:
#plot sales diff
plot_data = [
    go.Scatter(
        x=df_diff['RQST_DT'],
        y=df_diff['CNT'],
    )
]
plot_layout = go.Layout(
        title='ITService Count'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [344]:
#plot sales diff
plot_data = [
    go.Scatter(
        x=df_diff['RQST_DT'],
        y=df_diff['diff'],
    )
]
plot_layout = go.Layout(
        title='ITService Diff'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
pyoff.iplot(fig)

In [345]:
def fn_regression_adj_rsq(cols):
    model = smf.ols(formula=cols, data=df_supervised)
    # Fit the regression
    model_fit = model.fit()
    # Extract the adjusted r-squared
    regression_adj_rsq = model_fit.rsquared_adj
    
    return regression_adj_rsq

In [346]:
df_supervised = df_diff.drop(['prev_CNT'],axis=1)

#adding lags
for idx in range(1, 10):
    lagNum = idx * 10 + 1
    cols = 'diff~'
    for inc in range(1,lagNum):
        field_name = 'lag_' + str(inc)
        df_supervised[field_name] = df_supervised['diff'].shift(inc)
        if(inc == lagNum -1):
            cols = cols + field_name
        else:    
            cols = cols + field_name + '+'
    #drop null values
    df_supervised = df_supervised.dropna().reset_index(drop=True)    
    val = fn_regression_adj_rsq(cols)
    print(str(idx) + ' ' + str(val))    

1 0.3791745880944519
2 0.4876679746059025
3 0.450355422700926
4 0.4290307076058869
5 0.39630819205514844
6 nan


C:\Users\admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\regression\linear_model.py:1751: RuntimeWarning:

divide by zero encountered in true_divide

C:\Users\admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\statsmodels\regression\linear_model.py:1751: RuntimeWarning:

invalid value encountered in double_scalars



ValueError: zero-size array to reduction operation maximum which has no identity

In [347]:
#create dataframe for transformation from time series to supervised
df_supervised = df_diff.drop(['prev_CNT'],axis=1)
#adding lags
for inc in range(1,3):
    field_name = 'lag_' + str(inc)
    df_supervised[field_name] = df_supervised['diff'].shift(inc)
#drop null values
df_supervised = df_supervised.dropna().reset_index(drop=True)

In [349]:
df_supervised.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   RQST_DT  253 non-null    datetime64[ns]
 1   CNT      253 non-null    int64         
 2   diff     253 non-null    float64       
 3   lag_1    253 non-null    float64       
 4   lag_2    253 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 10.0 KB


In [350]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

# 날짜를 제외한 컬럼 가져오기
scale_cols = df_supervised.columns.values.tolist()[1:]

# 정규화 수행
scaled_df = scaler.fit_transform(df_supervised[scale_cols]) 

# numpy 리턴
print(type(scaled_df))

# 정규화된 데이터 프레임 생성
scaled_df = pd.DataFrame(scaled_df, columns=scale_cols)
print(scaled_df)


<class 'numpy.ndarray'>
          CNT      diff     lag_1     lag_2
0    0.147059  0.372093  0.651163  0.534884
1    0.147059  0.488372  0.372093  0.651163
2    0.000000  0.372093  0.488372  0.372093
3    0.117647  0.581395  0.372093  0.488372
4    0.117647  0.488372  0.581395  0.372093
..        ...       ...       ...       ...
248  0.029412  0.302326  0.581395  0.441860
249  0.147059  0.581395  0.302326  0.581395
250  0.058824  0.418605  0.581395  0.302326
251  0.117647  0.534884  0.418605  0.581395
252  0.058824  0.441860  0.534884  0.418605

[253 rows x 4 columns]


In [351]:

df_supervised.columns.values.tolist()[2:]

['diff', 'lag_1', 'lag_2']

In [352]:
# 입력데이터
feature_cols = df_supervised.columns.values.tolist()[2:]
## 정답데이터
label_cols = ['CNT']  

feature_df = pd.DataFrame(scaled_df, columns=feature_cols)
label_df = pd.DataFrame(scaled_df, columns=label_cols)

print(feature_df, label_df)

         diff     lag_1     lag_2
0    0.372093  0.651163  0.534884
1    0.488372  0.372093  0.651163
2    0.372093  0.488372  0.372093
3    0.581395  0.372093  0.488372
4    0.488372  0.581395  0.372093
..        ...       ...       ...
248  0.302326  0.581395  0.441860
249  0.581395  0.302326  0.581395
250  0.418605  0.581395  0.302326
251  0.534884  0.418605  0.581395
252  0.441860  0.534884  0.418605

[253 rows x 3 columns]           CNT
0    0.147059
1    0.147059
2    0.000000
3    0.117647
4    0.117647
..        ...
248  0.029412
249  0.147059
250  0.058824
251  0.117647
252  0.058824

[253 rows x 1 columns]


In [353]:
# 딥러닝 학습을 위한 넘파이로 변환
feature_np = feature_df.to_numpy()
label_np = label_df.to_numpy()

print(feature_np.shape, label_np.shape)

(253, 3) (253, 1)


In [354]:
def make_sequence_dataset(feature, label, window_size):
    
    feature_list = [] 
    label_list = []
    
    for i in range(len(feature) - window_size):
        
        feature_list.append(feature[i:i+window_size])
        label_list.append(label[i+window_size])
           #3차원 텐서 변환 
    return np.array(feature_list), np.array(label_list)

In [355]:
## 시계열 데이터 생성
window_size = 30

X, Y = make_sequence_dataset(feature_np, label_np, window_size)

print(X.shape, Y.shape)

(223, 30, 3) (223, 1)


In [357]:
## 데이터 나누기 
split = -50

x_train = X[0:split]
y_train = Y[0:split]

x_test = X[split:]
y_test = Y[split:]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(173, 30, 3) (173, 1)
(50, 30, 3) (50, 1)


In [362]:
model = Sequential()
model.add(LSTM(32, batch_input_shape=(1, x_train.shape[1], x_train.shape[2]), stateful=True))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(x_train, y_train, epochs=260, batch_size=1, verbose=1, shuffle=False)

Epoch 1/260
173/173 [==============================] - 1s 4ms/step - loss: 0.0336
Epoch 2/260
173/173 [==============================] - 1s 4ms/step - loss: 0.0270
Epoch 3/260
173/173 [==============================] - 1s 4ms/step - loss: 0.0281
Epoch 4/260
173/173 [==============================] - 1s 4ms/step - loss: 0.0281
Epoch 5/260
173/173 [==============================] - 1s 4ms/step - loss: 0.0292
Epoch 6/260
173/173 [==============================] - 1s 4ms/step - loss: 0.0298
Epoch 7/260
173/173 [==============================] - 1s 4ms/step - loss: 0.0300
Epoch 8/260
173/173 [==============================] - 1s 4ms/step - loss: 0.0309
Epoch 9/260
173/173 [==============================] - 1s 4ms/step - loss: 0.0325
Epoch 10/260
173/173 [==============================] - 1s 4ms/step - loss: 0.0327
Epoch 11/260
173/173 [==============================] - 1s 4ms/step - loss: 0.0330
Epoch 12/260
173/173 [==============================] - 1s 4ms/step - loss: 0.0331
Epoch 13/260


In [377]:
y_pred = model.predict(x_test,batch_size=1)

y_pred

array([[0.21886219],
       [0.12411024],
       [0.1790084 ],
       [0.12478296],
       [0.12343342],
       [0.10175511],
       [0.11781973],
       [0.10494059],
       [0.09281541],
       [0.12863001],
       [0.12428443],
       [0.10904795],
       [0.1186343 ],
       [0.10385133],
       [0.09367266],
       [0.1941897 ],
       [0.17034122],
       [0.09505349],
       [0.10863125],
       [0.10642923],
       [0.1094545 ],
       [0.11870056],
       [0.10051306],
       [0.10490735],
       [0.09749824],
       [0.10305677],
       [0.10444567],
       [0.10330112],
       [0.10865717],
       [0.10846312],
       [0.08661886],
       [0.10065094],
       [0.19665365],
       [0.12365881],
       [0.14445177],
       [0.15349351],
       [0.14937188],
       [0.15738304],
       [0.11338224],
       [0.1274128 ],
       [0.08359715],
       [0.1409535 ],
       [0.1741492 ],
       [0.12480618],
       [0.18495865],
       [0.1610378 ],
       [0.14363304],
       [0.182

In [371]:
#reshape y_pred
y_pred = y_pred.reshape(y_pred.shape[0], 1, y_pred.shape[1])
y_pred.shape



(50, 1, 1)